In [1]:
import os, sys
sys.path.append("..")

# Optimization of Pauli Poly

There are two measure to estimate performances of `PauliPoly`.

- Naive `PauliPoly` implementation.
- Qiskit `SparsePauliOp` and `PauliList`.
- 
The PauliPoly in the origial OptTrot library was a naive implementation written by python.
In here, we will optimize the routine as soon as possible using scipy and numba routines.

In [2]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly

import numpy as np
import matplotlib.pyplot as plt

In [3]:
from importlib import reload
import paulipoly
reload(paulipoly)
from paulipoly import PauliPoly as NewPauliPoly
import ten_con
reload(ten_con)
from ten_con import _mat_to_coef_mat, _coef_to_mat

In [4]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

In [5]:
n = 3
plist1 = [
    PauliElement(3, 5, n, 3),
    PauliElement(3, 1, n, 3),
    PauliElement(0, 4, n, 3),
    PauliElement(4, 2, n, 3),
]
plist2 = [
    PauliElement(2, 3, n, 3),
    PauliElement(2, 0, n, 3),
    PauliElement(0, 3, n, 3),
    PauliElement(3, 1, n, 3),
]
plist3 = [
    PauliElement(1, 1, n, 1),
    PauliElement(3, 0, n, 1),
    PauliElement(2, 2, n, 1),
    PauliElement(2, 5, n, 1),
    PauliElement(4, 4, n, 1),
    PauliElement(3, 5, n, 1),
    PauliElement(3, 1, n, 1),
]

In [39]:
qubits = 6
ndim = int(2**qubits)
A = np.random.rand(ndim,ndim)
B = np.random.rand(ndim,ndim)
A[A<0.3] = 0.
B[B<0.9] = 0.
test_ppoly = PauliPoly.from_coef_mat(A)

In [40]:
plist = list(test_ppoly._terms.values())
coef_mat = test_ppoly.coef_matrix
mat = test_ppoly.matrix

## Coef matrix to PauliPoly

In [41]:
from opttrot.pauli_c import _bignum_bytes, get_paulilist_from_coefs
from scipy.sparse import coo_matrix

In [42]:
cmat = coo_matrix(coef_mat)

In [43]:
%%timeit
PauliPoly.from_coef_mat(coef_mat) # Naive

101 ms ± 3.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit
NewPauliPoly.from_coef_mat(coef_mat) # scipy sparse routine.

46 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Hermit matrix to PauliPoly

In [45]:
%%timeit
PauliPoly.from_matrix(mat)

In [ ]:
%%timeit
NewPauliPoly.from_matrix(mat)

226 μs ± 4.41 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
SparsePauliOp.from_operator(mat)

9.88 μs ± 232 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Addition

In [ ]:
# Naive
pp001 = PauliPoly.from_coef_mat(A)
pp002 = PauliPoly.from_coef_mat(B)
# Improved
npp001 = NewPauliPoly.from_coef_mat(A)
npp002 = NewPauliPoly.from_coef_mat(B)

# Qiskit
spa001 = SparsePauliOp.from_operator(A)
spa002 = SparsePauliOp.from_operator(B)


**Comparsion: Matrix and set methods**

Qubits = 5

- Matrix: 58.3 ms ± 2.4 ms
- Set: 13.2 ms ± 111 µs

Qubits = 8

- Matrix: 3.68 s ± 67 ms per loop
- Set: 2 s ± 74.5 ms

In [ ]:
%%timeit
pp001+pp002

467 μs ± 4.39 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
npp001+npp002

280 μs ± 3.81 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
spa001+spa002

11.1 μs ± 67.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Mat multiplication

In [ ]:
%%timeit
pp001@pp002

599 μs ± 3.46 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
npp001@npp002

287 μs ± 16.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
spa001@spa002

25.3 μs ± 781 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
